In [10]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
lfw_dir = 'dataset/lfw_funneled'
pairs_file = 'dataset/pairsDevTest.txt'

with open(pairs_file, 'r') as f:
    lines = f.readlines()[1:]

pairs = []
labels = []

for line in lines:
    parts = line.strip().split()
    if len(parts) == 3:
        # positive pair: name, idx1, idx2
        name, idx1, idx2 = parts
        img1 = os.path.join(lfw_dir, name, f"{name}_{int(idx1):04d}.jpg")
        img2 = os.path.join(lfw_dir, name, f"{name}_{int(idx2):04d}.jpg")
        label = 1
    elif len(parts) == 4:
        # negative pair: name1, idx1, name2, idx2
        name1, idx1, name2, idx2 = parts
        img1 = os.path.join(lfw_dir, name1, f"{name1}_{int(idx1):04d}.jpg")
        img2 = os.path.join(lfw_dir, name2, f"{name2}_{int(idx2):04d}.jpg")
        label = 0
    else:
        print(f"Invalid line: {parts}")
        break

    if os.path.exists(img1) and os.path.exists(img2):
        pairs.append((img1, img2))
        labels.append(label)
    else:
        print(f"Invalid line: {parts}")
        break

pair_labels = [(pairs[i], labels[i]) for i in range(len(pairs))]
print(f"Загружено {len(pairs)} пар")


Загружено 1000 пар
